In [9]:
from datetime import datetime
#import pandas as pd
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from datetime import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
# get filepath to song data file
song_data = "s3a://udacity-dend/song_data"
output_data = "s3a://udacity-project-bucket/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1589247620637_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# read song data file
df_song = spark.read.json(song_data + "/*/*/*")
#df_song.limit(1).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# extract columns to create songs table
songs_table = df_song[['song_id', 'title', 'artist_id', 'year', 'duration']]  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# write songs table to parquet files partitioned by year and artist
songs_table.dropDuplicates().write.partitionBy("year","artist_id").parquet(output_data + "songs.parquet")

In [11]:
# extract columns to create artists table
artists_table = df_song[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# write artists table to parquet files
artists_table.dropDuplicates().write.parquet(output_data + "artists.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# get filepath to song data file
log_data = "s3a://udacity-dend/log_data"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# read log data file
df = spark.read.json(log_data + "/*/*/*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# filter by actions for song plays

# explanation: song plays needs all 'NextSong' results
df = df.filter(df.page=='NextSong')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# extract columns for users table    
users_table = df[['userId', 'firstName', 'lastName', 'gender', 'level']]  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# write users table to parquet files
users_table.dropDuplicates().write.parquet(output_data + "user.parquet")
#df.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# create timestamp column from original timestamp column
get_datetime = udf(lambda x: datetime.fromtimestamp( (x/1000.0) ), T.TimestampType()) 
df = df.withColumn("start_time", get_datetime(df.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df.createOrReplaceTempView("log_staging_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# create columns for time table
time_table = spark.sql('''
    SELECT start_time,
    EXTRACT(hour from start_time) as hour,
    EXTRACT(day from start_time) as day,
    EXTRACT(week from start_time) as week,
    EXTRACT(month from start_time) as month,
    EXTRACT(year from start_time) as year,
    DAYOFWEEK(start_time) as weekday
    
    from log_staging_table
''').collect()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 46426)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 266, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 254, in authenticate_and_accum_updates
    received_to

In [17]:
# write time table to parquet files partitioned by year and month
#print(type(users_table))
print(type(time_table))
time_table_dataframe = spark.createDataFrame(time_table)
print(type(time_table_dataframe))
#time_table_dataframe.limit(1).toPandas()


time_table_dataframe.dropDuplicates().write.partitionBy("year","month").parquet(output_data + "time.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'list'>
<class 'pyspark.sql.dataframe.DataFrame'>

In [19]:

# read in song data to use for songplays table
song_df =  spark.read.json(song_data + "/*/*/*")
song_df.createOrReplaceTempView("songs_staging_table")

# extract columns from joined song and log datasets to create songplays table 
temp_table = spark.sql('''
    SELECT 
    
        a.start_time, a.userId, a.level, b.song_id,
        b.artist_id, a.sessionId, a.location, a.userAgent,
        EXTRACT(month from a.start_time) as month,
        EXTRACT(year from a.start_time) as year
    from log_staging_table a
    inner join songs_staging_table b on a.song = b.title
''').collect()

songplays_table = spark.createDataFrame(temp_table)
#songplays_table.limit(1).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# write songplays table to parquet files partitioned by year and month
songplays_table.dropDuplicates().write.partitionBy("year","month").parquet(output_data + "songplays.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…